In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Get the Data
** Read the 'KNN_Project_Data csv file into a dataframe **

In [2]:
df = pd.read_csv("hawaii.csv")


In [3]:
df.columns

Index(['loan_type', 'property_type', 'purpose', 'occupancy', 'amount',
       'status', 'sex', 'income'],
      dtype='object')

**Check the head of the dataframe.**

In [4]:
df.head()

,loan_type,property_type,purpose,occupancy,amount,status,sex,income
0,Conventional,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,396,Loan originated,Female,152.0
1,VA-guaranteed,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,662,Loan originated,Male,111.0
2,Conventional,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,461,Loan originated,Female,138.0
3,Conventional,One-to-four family dwelling (other than manufa...,Refinancing,Not owner-occupied as a principal dwelling,345,Loan originated,Female,184.0
4,Conventional,One-to-four family dwelling (other than manufa...,Refinancing,Not owner-occupied as a principal dwelling,210,Application withdrawn by applicant,Female,99.0


In [5]:
df.columns

Index(['loan_type', 'property_type', 'purpose', 'occupancy', 'amount',
       'status', 'sex', 'income'],
      dtype='object')

# EDA

In [6]:
# Remove null values
df = df.dropna()

### Status

In [7]:
# Keep only records that either granted or rejected loans
states = ['originated', 'approved', 'denied']
df = df[df.apply(lambda x: any(s in x.status for s in states), axis=1)]

In [8]:
df.status.value_counts()

Loan originated                                25429
Application denied by financial institution     5321
Application approved but not accepted           1164
Name: status, dtype: int64

In [9]:
# Merged approved and withdrawn in row status
def f(row):
    return 1 if ("approved" in row.status or "originated" in row.status) else 0
df['status'] = df.apply(f, axis=1)


In [10]:
df['status'].unique()

array([1, 0])

In [11]:
# Making sure the counts added up
df.status.value_counts()

1    26593
0     5321
Name: status, dtype: int64

# Encode Categorical Features

In [12]:
df.dtypes

loan_type         object
property_type     object
purpose           object
occupancy         object
amount             int64
status             int64
sex               object
income           float64
dtype: object

In [13]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(df)
categorical_columns


['loan_type', 'property_type', 'purpose', 'occupancy', 'sex']

In [14]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(df)

# Standardize the Variables

Time to standardize the variables.

** Import StandardScaler from Scikit learn.**

In [15]:
# from sklearn.preprocessing import StandardScaler

** Create a StandardScaler() object called scaler.**

In [16]:
# scaler = StandardScaler()

** Fit scaler to the features.**

In [17]:
# scaler.fit(df.drop('status', axis=1))

**Use the .transform() method to transform the features to a scaled version.**

In [18]:
# scaled_features = scaler.transform(df.drop('status', axis=1))


**Convert the scaled features to a dataframe and check the head of this dataframe to make sure the scaling worked.**

In [19]:
# df_features = pd.DataFrame(scaled_features, columns=[col for col in df.columns if col != 'status'])
# df_features.head()


# Train Test Split

**Use train_test_split to split your data into a training set and a testing set.**

In [20]:
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('status', axis=1), df['status'], test_size=0.3, random_state=101)

from sklearn.preprocessing import OneHotEncoder

# Create the encoder.
encoder = OneHotEncoder(handle_unknown="ignore")
encoder.fit(X_train)    # Assume for simplicity all features are categorical.

# Apply the encoder.
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)


In [44]:
X_train.shape

(22339, 2228)

In [45]:
y_train.shape

(22339,)

# Using KNN

**Import KNeighborsClassifier from scikit learn.**

In [23]:
from sklearn.neighbors import KNeighborsClassifier

**Create a KNN model instance with n_neighbors=1**

In [24]:
knn = KNeighborsClassifier(n_neighbors=15)

**Fit this KNN model to the training data.**

In [25]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=15)

# Predictions and Evaluations
Let's evaluate our KNN model!

**Use the predict method to predict values using your KNN model and X_test.**

In [26]:
prediction = knn.predict(X_test)

** Create a confusion matrix and classification report.**

In [27]:
from sklearn.metrics import confusion_matrix, classification_report

In [28]:
print(confusion_matrix(y_test, prediction))

[[  48 1519]
 [  87 7921]]


In [29]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.36      0.03      0.06      1567
           1       0.84      0.99      0.91      8008

    accuracy                           0.83      9575
   macro avg       0.60      0.51      0.48      9575
weighted avg       0.76      0.83      0.77      9575



# Choosing a K Value
Let's go ahead and use the elbow method to pick a good K Value!

** Create a for loop that trains various KNN models with different k values, then keep track of the error_rate for each of these models with a list. Refer to the lecture if you are confused on this step.**

In [30]:
# error_rate = []
# for i in range(1, 50):
#     print(i)
#     knn = KNeighborsClassifier(n_neighbors=i)
#     knn.fit(X_train, y_train)
#     prediction = knn.predict(X_test)
#     error_rate.append(np.mean(prediction != y_test))

**Now create the following plot using the information from your for loop.**

In [31]:
# figure = plt.figure(figsize=(7,5))
# plt.plot(np.arange(1,50), error_rate, marker='o')
# plt.title('Error rate vs K Value')
# plt.xlabel('K Value')
# plt.ylabel('Error rate')

## Retrain with new K Value

**Retrain your model with the best K value (up to you to decide what you want) and re-do the classification report and the confusion matrix.**

In [32]:
knn = KNeighborsClassifier(n_neighbors=35)
knn.fit(X_train, y_train)
k_100_prediction = knn.predict(X_test)
print(confusion_matrix(y_test, k_100_prediction))


[[  11 1556]
 [  31 7977]]


In [33]:
print(classification_report(y_test, k_100_prediction))


              precision    recall  f1-score   support

           0       0.26      0.01      0.01      1567
           1       0.84      1.00      0.91      8008

    accuracy                           0.83      9575
   macro avg       0.55      0.50      0.46      9575
weighted avg       0.74      0.83      0.76      9575



In [34]:
knn.score(X_test, y_test)

0.8342558746736293

In [36]:
import pickle
knnPickle = open('model', 'wb')

# source, destination
pickle.dump(knn, knnPickle)

# close the file
knnPickle.close()
